In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score
import mne
from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf,read_raw_edf,read_raw_gdf
from mne.datasets import eegbci
from mne.decoding import CSP

In [2]:
def get_data():
    path = "dataset\\bci_dataset\\train"
    files = os.listdir(path)
    data_path = []
    for f in files:
        file = os.path.join(path,f)
        data_path.append(file)
    
    tmin, tmax = -0.5, 4.
    event_id = dict(left=1, right = 2, foot=3,tongue=4)

    raw = concatenate_raws([read_raw_gdf(f, preload=True) for f in data_path])

    # strip channel names of "." characters
    raw.rename_channels(lambda x: x.strip('.'))

    # Apply band-pass filter
    #raw.filter(7., 30., fir_design='firwin', skip_by_annotation='edge')

    events, _ = events_from_annotations(raw)
    
    picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                       exclude='bads')
    #delete eog band
    picks = np.delete(picks,[22,23,24])
    # Read epochs (train will be done only between 1 and 2s)
    # Testing will be done with a running classifier
    epochs = Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks,
                    baseline=None, preload=True)
    epochs.crop(tmin=1., tmax=None)
    labels = epochs.events[:, 2] -1
    return epochs.get_data()[:, :, :256], labels


epochs_data, labels = get_data()

Extracting EDF parameters from c:\Users\asus\Desktop\Motorimagery_for_gamification\dataset\bci_dataset\train\A01T.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 672527  =      0.000 ...  2690.108 secs...


C:\Users\asus\anaconda3\envs\tf_gpu\lib\site-packages\mne\io\edf\edf.py:1047: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
<ipython-input-2-8dc407ed6195>:12: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  raw = concatenate_raws([read_raw_gdf(f, preload=True) for f in data_path])


Extracting EDF parameters from c:\Users\asus\Desktop\Motorimagery_for_gamification\dataset\bci_dataset\train\A02T.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 677168  =      0.000 ...  2708.672 secs...


C:\Users\asus\anaconda3\envs\tf_gpu\lib\site-packages\mne\io\edf\edf.py:1047: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
<ipython-input-2-8dc407ed6195>:12: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  raw = concatenate_raws([read_raw_gdf(f, preload=True) for f in data_path])


Extracting EDF parameters from c:\Users\asus\Desktop\Motorimagery_for_gamification\dataset\bci_dataset\train\A03T.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 660529  =      0.000 ...  2642.116 secs...


...

Extracting EDF parameters from c:\Users\asus\Desktop\Motorimagery_for_gamification\dataset\bci_dataset\train\A08T.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 675269  =      0.000 ...  2701.076 secs...


C:\Users\asus\anaconda3\envs\tf_gpu\lib\site-packages\mne\io\edf\edf.py:1047: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
<ipython-input-2-8dc407ed6195>:12: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  raw = concatenate_raws([read_raw_gdf(f, preload=True) for f in data_path])


Extracting EDF parameters from c:\Users\asus\Desktop\Motorimagery_for_gamification\dataset\bci_dataset\train\A09T.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 673327  =      0.000 ...  2693.308 secs...


C:\Users\asus\anaconda3\envs\tf_gpu\lib\site-packages\mne\io\edf\edf.py:1047: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
<ipython-input-2-8dc407ed6195>:12: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  raw = concatenate_raws([read_raw_gdf(f, preload=True) for f in data_path])


Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']
Not setting metadata
Not setting metadata
289 matching events found
No baseline correction applied
0 projection items activated
Loading data for 289 events and 1126 original time points ...
9 bad epochs dropped


In [3]:
# 280 events
# 22 channel
# 256 Time
print(epochs_data.shape)
print(labels.shape)
print(epochs_data.shape[1])
print(labels)

(280, 22, 256)
(280,)
22
[3 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 3 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 3 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 3 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 3 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [4]:
def amplitude(x):
    """Data augmentation for adding noise to amplitude of sepctral image
    ----------
    X : array, shape (n_channels, n_times)
        The input signals.
    Returns
    -------
    X_t : array, shape (n_channels, n_times)
        reconstructed new time series from inverse STFT
    """
    print("Hello")



In [5]:
import torch   
import torch.optim as optim  
from torch.utils.data import Dataset, DataLoader  
from torch.utils.data import Subset  
from torch import nn  
import torch.nn.functional as F  
from torch.utils.data import RandomSampler  
from torch.utils.data import SequentialSampler  

from common import EpochsDataset  

cv = ShuffleSplit(10, test_size=0.2, random_state=42)
cv_split = cv.split(epochs_data)
train_idx, test_idx = next(cv_split)


def scale(X):
    """Standard scaling of data along the last dimention.
    Parameters
    ----------
    X : array, shape (n_channels, n_times)
        The input signals.
    Returns
    -------
    X_t : array, shape (n_channels, n_times)
        The scaled signals.
    """
    return X / 2e-5

dataset = EpochsDataset(epochs_data, labels, transform=scale)

ds_train, ds_valid = Subset(dataset, train_idx), Subset(dataset, test_idx)

batch_size_train = len(ds_train)
batch_size_valid = len(ds_valid)
sampler_train = RandomSampler(ds_train)
sampler_valid = SequentialSampler(ds_valid)

# create loaders
num_workers = 0
loader_train = \
    DataLoader(ds_train, batch_size=batch_size_train,
               num_workers=num_workers, sampler=sampler_train)
loader_valid = \
    DataLoader(ds_valid, batch_size=batch_size_valid,
               num_workers=num_workers, sampler=sampler_valid)

In [6]:
class SingleNet(nn.Module):
    def __init__(self):
        super().__init__()

        # define model architecture
        # torch.Size([36, 1, 64, 256])
        # temporal convolutional kernel 25 filter with size(11,1)
        #input [224, 1, 22, 256]
        #A block
        self.temporalConv = nn.Conv1d(in_channels=1,out_channels=8,kernel_size=(11,1),stride=1)
        self.spatialConv = nn.Conv1d(in_channels=8,out_channels=8,kernel_size=(12,1),stride=1)
        self.maxPooling = nn.MaxPool2d((1,3))
        
        #B block
        # 1st convo output [25,1,371]
        #input [224, 8, 1, 85]
        self.firstConv = nn.Conv1d(in_channels=8, out_channels=8,kernel_size=(1,1))
        self.secondConv = nn.Conv1d(in_channels=8, out_channels=33,kernel_size=(1,1))
        self.thirdConv = nn.Conv1d(in_channels=33,out_channels=33,kernel_size=(1,1))
        self.maxPooling2 = nn.MaxPool2d((1,3))
        
        #C block
        #input [224, 41, 1, 28]
        self.classify = nn.Conv1d(in_channels=41,out_channels=41,kernel_size=(1,11))
        self.maxPooling3 = nn.MaxPool2d((1,3))
        self.flatten = nn.Flatten()
        self.elu = nn.ELU()
        self.fully = nn.Linear(246,4)
        
        
    def forward(self, x):
        #Conv1d(1,ch,kernel_size=10) on tensor [batch=1, channels=1, time=400]
        x = self.temporalConv(x)
        x= self.spatialConv(x)
        x = self.elu(x)
        x = self.maxPooling(x)
        
        out2 = self.firstConv(x)
        out2 = self.secondConv(out2)
        out2 = self.thirdConv(out2)
        out3 = torch.cat((x,out2),1)
        out3 = self.maxPooling2(out3)
        
        out3 = self.classify(out3)
        out3 = self.maxPooling3(out3)
        out3 = self.flatten(out3)
        out3 = self.elu(out3)
        out3 = self.fully(out3)
        out3 = self.elu(out3)
        out3 = F.log_softmax(out3, dim=1)
        
        return out3

In [7]:
#device = 'cuda'
device = 'cpu'
model = SingleNet()


In [8]:
# Train
from common import train

lr = 1e-4
n_epochs = 50
patience = 100

model.to(device=device) 
optimizer = optim.Adam(model.parameters(), lr=lr,weight_decay=1e-4)

train(model, loader_train, loader_valid, optimizer, n_epochs, patience, device)


Starting epoch 1 / 50


  0%|          | 0/1 [00:00<?, ?it/s]C:\Users\asus\anaconda3\envs\tf_gpu\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
avg val loss: 1.3289: 100%|██████████| 1/1 [00:00<00:00, 29.49it/s]


---  Accuracy : 0.7678571343421936 


best val loss inf -> 1.3289

Starting epoch 2 / 50


avg val loss: 1.3031: 100%|██████████| 1/1 [00:00<00:00, 40.11it/s]


---  Accuracy : 0.9285714030265808 


best val loss 1.3289 -> 1.3031

Starting epoch 3 / 50


...

avg val loss: 0.3241: 100%|██████████| 1/1 [00:00<00:00, 47.75it/s]


---  Accuracy : 0.9642857313156128 


best val loss 0.3338 -> 0.3241

Starting epoch 50 / 50


avg val loss: 0.3151: 100%|██████████| 1/1 [00:00<00:00, 50.13it/s]

---  Accuracy : 0.9642857313156128 


best val loss 0.3241 -> 0.3151


SingleNet(
  (temporalConv): Conv1d(1, 8, kernel_size=(11, 1), stride=(1,))
  (spatialConv): Conv1d(8, 8, kernel_size=(12, 1), stride=(1,))
  (maxPooling): MaxPool2d(kernel_size=(1, 3), stride=(1, 3), padding=0, dilation=1, ceil_mode=False)
  (firstConv): Conv1d(8, 8, kernel_size=(1, 1), stride=(1,))
  (secondConv): Conv1d(8, 33, kernel_size=(1, 1), stride=(1,))
  (thirdConv): Conv1d(33, 33, kernel_size=(1, 1), stride=(1,))
  (maxPooling2): MaxPool2d(kernel_size=(1, 3), stride=(1, 3), padding=0, dilation=1, ceil_mode=False)
  (classify): Conv1d(41, 41, kernel_size=(1, 11), stride=(1,))
  (maxPooling3): MaxPool2d(kernel_size=(1, 3), stride=(1, 3), padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (elu): ELU(alpha=1.0)
  (fully): Linear(in_features=246, out_features=4, bias=True)
)

In [9]:
# Test model works:

n_samples_test = 10
y_test = torch.randint(0, 2, (n_samples_test,))
y_pred = model.forward(torch.randn(n_samples_test, 1, *epochs_data.shape[1:]))
output = F.nll_loss(y_pred, y_test)
_, top_class = y_pred.topk(1, dim=1)


In [10]:
print(top_class)

tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]])
